In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# importando pacotes para selecao de features para o modelo
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from scipy.stats import ttest_ind, chi2_contingency
import statsmodels.api as sm
from imblearn.over_sampling import SMOTE

In [ ]:
# importando alguns algoritmos de treinamento de modelos de classificacao
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as Tree
from sklearn.ensemble import RandomForestClassifier as Forest
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [ ]:
# importanto metricas do sklearn para avalicao dos modelos
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

## Readding prepared dataset

In [ ]:
#df = pd.read_csv('clean_dataset_v1.csv') # versao sem valores nulos, porem, tem outliers
df = pd.read_csv('clean_dataset_v2.csv') # versao sem outliers
df.shape

In [ ]:
df.head(8)

In [ ]:
df.columns

# Selecao de Atributos

In [ ]:
X = df.drop('TenYearCHD', axis=1)
Y = df[['TenYearCHD']]

In [ ]:
X=X.drop(['Unnamed: 0'], axis=1)

In [ ]:
model_lr = LogisticRegression()
rfecv = RFECV(estimator = model_lr,
             step=1,
             cv=5).fit(X, Y) 

In [ ]:
X.columns

In [ ]:
rfecv.support_

In [ ]:
selected_features = [X.columns[i] for i in range(len(rfecv.support_)) if rfecv.support_[i]]

In [ ]:
selected_features

### Split the dataset

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X[selected_features], Y, test_size=0.3, random_state=42)

In [ ]:
X[selected_features].describe()

In [ ]:
sns.countplot(Y['TenYearCHD'].astype('category'))
plt.show()

### Aplicando o Oversampling no atributo desbalanceado

In [ ]:
smote = SMOTE(random_state=1)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

In [ ]:
sns.countplot(Y_train['TenYearCHD'].astype('category'))
plt.show()

## Treinamento do modelo

In [ ]:
models = {
    "KNN": KNN('algorithm': 'auto', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'), #('algorithm': 'auto', 'n_neighbors': 10, 'p': 1, 'weights': 'distance')
    "Decision Tree": Tree('criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5 ), #'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5 
    "Random Forest": Forest(), 
    "SVM": SVC()
} 

In [326]:
def find_params(name, model, params):
    
     # Configurar o GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy')
    
    # Ajustar o GridSearchCV
    grid_search.fit(X_train, Y_train)

    print ("Modelo: ", name)
    # Melhor combinação de parâmetros
    print(f"Melhores parâmetros: {grid_search.best_params_}")
    
    # Melhor modelo
    best_model = grid_search.best_estimator_
    
    # Avaliar o modelo
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(Y_test, y_pred)
    print(f"Acurácia do modelo ajustado: {accuracy:.2f}")

In [335]:
find_params("KNN", KNN(), {
            'n_neighbors': [3, 5, 7, 10],
            'weights': ['uniform', 'distance'],
            'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            'p': [1, 2]
        })

/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

Modelo:  KNN
Melhores parâmetros: {'algorithm': 'auto', 'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Acurácia do modelo ajustado: 0.70


/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [330]:
find_params("Decision Tree", Tree(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': [None, 'sqrt', 'log2']
        })   

Modelo:  Decision Tree
Melhores parâmetros: {'criterion': 'gini', 'max_depth': 30, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
Acurácia do modelo ajustado: 0.75


In [331]:
find_params("Random Forests", Forest(), {
            'n_estimators': [50, 100, 200],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': [None, 'sqrt', 'log2']
        })


/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/pacote/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

KeyboardInterrupt: 

In [ ]:

find_params("SVM", SVC(), {'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'],
            'degree': [3, 4, 5]  # Usado apenas com kernel='poly'
        })

In [ ]:
for label, model in models.items():
    model.fit(X_train[selected_features], Y_train)

In [ ]:
for label, model in models.items():    
    print(label, model.predict([X_test[selected_features].iloc[10]]))

In [ ]:
Y_test.iloc[10]

## Avaliacao do modelo

#### matriz de confusao
Expoe a quantidade de verdadeiros positivos, verdadeiros negativos, falsos positivos e falsos negativos


In [ ]:
for label, model in models.items():    
    Y_pred = model.predict(X_test)
    print("=======Confusion Matrix: ", label)
    print(confusion_matrix(Y_test, Y_pred))
    print("=================================\n")

### Acuracia

Representa a proporcao das prrevisoes correctas (tanto positivas quanto negativas) em relacao ao total de previsoes

In [ ]:
for label, model in models.items():    
    Y_pred = model.predict(X_test)
    print("=======Acuracia: ", label)
    print(accuracy_score(Y_test, Y_pred, normalize=True)*100, "%")
    print("=================================\n")

### Precisao do Modelo

Representa a proporção de previsões positivas corretas em relação ao total de previsões positivas
>Importante quando o custo de falsos positivos é alto

In [ ]:
for label, model in models.items():    
    Y_pred = model.predict(X_test)
    print("=======Precisao: ", label)
    print(precision_score(Y_test, Y_pred)*100, "%")
    print("=================================\n")